In [52]:
import tensorflow.compat.v2 as tf
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import Dropout
# from keras import regularizers
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import time
import h5py
import kerastuner as kt
import tempfile
import os
import sys
import csv
sys.path.append('/Users/jackkelly/Desktop/Building_Model/qkeras')
from qkeras.utils import model_quantize
from qkeras.utils import model_save_quantized_weights
from qkeras.utils import print_model_sparsity
import qkeras
import tensorflow_model_optimization as tfmot

In [3]:
# Reproducibility
seed = 9 # arbitrary number
np.random.seed(seed=seed)
tf.random.set_seed(seed=seed)

In [4]:
### IMPORT, RAMDOMIZE AND SCALE DATA ###
def processing_data(N_channels, file_path):
    df = pd.read_csv(file_path)
    # dataset
    dataset = df.values
    # input power and channels statuses
    X = dataset[:,0:N_channels+1]
    # channel SNRs
    Y = dataset[:,N_channels+1:2*N_channels+1]
    # shuffle necessary because while shuffle=True in .fit() shuffles the data, it splits into different datasets first
    X, Y = shuffle(X, Y)
    # scaling between 0 and 1
    min_max_scaler = preprocessing.MinMaxScaler()
    X = min_max_scaler.fit_transform(X)
    Y = min_max_scaler.fit_transform(Y)
    return X, Y

In [5]:
### IMPORT 10,000 PIECES OF DATA ###

N_channels = 40
file_path = '/Users/jackkelly/Desktop/Building_Model/Data_NEW/10000_initial_data.csv'
X_scale, Y_scale = processing_data(N_channels, file_path)

In [5]:
### FUNCTION TAKES HYPERPARAMETERS AND RETURNS COMPILED MODEL ###

def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.InputLayer(input_shape=N_channels+1))
#     hp_regularizer = hp.Choice('regularizer_penalty', values=[1e-4, 1e-5, 1e-6])
#     for i in range(hp.Int('n_layers', 1, 2)):
    model.add(keras.layers.Dense(hp.Int('hidden_units', min_value=20, max_value=150, step=2), kernel_regularizer=tf.keras.regularizers.l2(1e-6)))
#     model.add(keras.layers.Dense(hp.Int('hidden_units', min_value=10, max_value=320, step=1), kernel_regularizer=tf.keras.regularizers.l2(hp_regularizer)))
#         model.add(keras.layers.Dense(hp.Int(f'hidden_{i}_units', min_value=10, max_value=320, step=1)))
#     SHOULD DROPOUT BE INCLUDED??
#     model.add(keras.layers.Dropout(0.1))
    model.add(keras.layers.Dense(N_channels, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(1e-6)))
#     model.add(keras.layers.Dense(N_channels, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(hp_regularizer)))
    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                    loss=keras.losses.MeanSquaredError(), metrics=['mean_squared_error', 'mean_absolute_error'])
    return model

In [6]:
### CREATE HYPERBAND TUNER ###

# The maximum number of epochs to train one model. It is recommended to 
# set this to a value slightly higher than the expected time 
# to convergence for your largest Model, and to use early stopping 
# during training
tuner = kt.Hyperband(model_builder,
                     objective=kt.Objective('val_mean_squared_error', 'min'),
                     max_epochs=50,
                     factor=3,
                     directory='/Users/jackkelly/Desktop/Building_Model/',
                     project_name='intro_to_kt_layers_fpga',
                     overwrite=True,
                     seed=8)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_mean_squared_error', patience=5)

In [7]:
### SEARCHING FOR THE ANN LAYOUT THAT ACCOMPLISHES THE HYPERPARAMTER GOALS ###

start_time = time.time()
tuner.search(X_scale, Y_scale, validation_split=0.2, callbacks=[stop_early], verbose=2)
searching_time = time.time() - start_time
print(searching_time)

Train on 8000 samples, validate on 2000 samples
Epoch 1/2
8000/8000 - 2s - loss: 0.1552 - mean_squared_error: 0.1551 - mean_absolute_error: 0.3615 - val_loss: 0.0950 - val_mean_squared_error: 0.0949 - val_mean_absolute_error: 0.2776
Epoch 2/2
8000/8000 - 1s - loss: 0.0639 - mean_squared_error: 0.0637 - mean_absolute_error: 0.2169 - val_loss: 0.0420 - val_mean_squared_error: 0.0418 - val_mean_absolute_error: 0.1712


Train on 8000 samples, validate on 2000 samples
Epoch 1/2
8000/8000 - 1s - loss: 0.1282 - mean_squared_error: 0.1281 - mean_absolute_error: 0.3239 - val_loss: 0.0598 - val_mean_squared_error: 0.0596 - val_mean_absolute_error: 0.2150
Epoch 2/2
8000/8000 - 1s - loss: 0.0321 - mean_squared_error: 0.0319 - mean_absolute_error: 0.1529 - val_loss: 0.0151 - val_mean_squared_error: 0.0148 - val_mean_absolute_error: 0.1065


Train on 8000 samples, validate on 2000 samples
Epoch 1/2
8000/8000 - 1s - loss: 0.1339 - mean_squared_error: 0.1338 - mean_absolute_error: 0.3323 - val_loss: 0.0673 - val_mean_squared_error: 0.0671 - val_mean_absolute_error: 0.2298
Epoch 2/2
8000/8000 - 1s - loss: 0.0386 - mean_squared_error: 0.0384 - mean_absolute_error: 0.1680 - val_loss: 0.0194 - val_mean_squared_error: 0.0191 - val_mean_absolute_error: 0.1197


Train on 8000 samples, validate on 2000 samples
Epoch 1/2
8000/8000 - 1s - loss: 0.0493 - mean_squared_error: 0.0490 - mean_absolute_error: 0.1560 - val_loss: 0.0121 - val_mean_squared_error: 0.0116 - val_mean_absolute_error: 0.0674
Epoch 2/2
8000/8000 - 1s - loss: 0.0106 - mean_squared_error: 0.0101 - mean_absolute_error: 0.0596 - val_loss: 0.0109 - val_mean_squared_error: 0.0103 - val_mean_absolute_error: 0.0575


Train on 8000 samples, validate on 2000 samples
Epoch 1/2
8000/8000 - 1s - loss: 0.2190 - mean_squared_error: 0.2189 - mean_absolute_error: 0.4434 - val_loss: 0.2037 - val_mean_squared_error: 0.2036 - val_mean_absolute_error: 0.4236
Epoch 2/2
8000/8000 - 1s - loss: 0.1941 - mean_squared_error: 0.1940 - mean_absolute_error: 0.4115 - val_loss: 0.1861 - val_mean_squared_error: 0.1860 - val_mean_absolute_error: 0.4005


...

Train on 8000 samples, validate on 2000 samples
Epoch 1/50
8000/8000 - 1s - loss: 0.2193 - mean_squared_error: 0.2192 - mean_absolute_error: 0.4432 - val_loss: 0.2050 - val_mean_squared_error: 0.2049 - val_mean_absolute_error: 0.4261
Epoch 2/50
8000/8000 - 1s - loss: 0.1961 - mean_squared_error: 0.1960 - mean_absolute_error: 0.4141 - val_loss: 0.1902 - val_mean_squared_error: 0.1902 - val_mean_absolute_error: 0.4050
Epoch 3/50
8000/8000 - 1s - loss: 0.1825 - mean_squared_error: 0.1824 - mean_absolute_error: 0.3967 - val_loss: 0.1771 - val_mean_squared_error: 0.1770 - val_mean_absolute_error: 0.3904
Epoch 4/50
8000/8000 - 1s - loss: 0.1693 - mean_squared_error: 0.1692 - mean_absolute_error: 0.3818 - val_loss: 0.1639 - val_mean_squared_error: 0.1638 - val_mean_absolute_error: 0.3750
Epoch 5/50
8000/8000 - 1s - loss: 0.1559 - mean_squared_error: 0.1559 - mean_absolute_error: 0.3661 - val_loss: 0.1506 - val_mean_squared_error: 0.1505 - val_mean_absolute_error: 0.3588
Epoch 6/50
8000/8000 -

Epoch 45/50
8000/8000 - 0s - loss: 0.0035 - mean_squared_error: 0.0031 - mean_absolute_error: 0.0461 - val_loss: 0.0035 - val_mean_squared_error: 0.0030 - val_mean_absolute_error: 0.0453
Epoch 46/50
8000/8000 - 0s - loss: 0.0033 - mean_squared_error: 0.0028 - mean_absolute_error: 0.0440 - val_loss: 0.0032 - val_mean_squared_error: 0.0028 - val_mean_absolute_error: 0.0433
Epoch 47/50
8000/8000 - 0s - loss: 0.0031 - mean_squared_error: 0.0026 - mean_absolute_error: 0.0421 - val_loss: 0.0030 - val_mean_squared_error: 0.0025 - val_mean_absolute_error: 0.0412
Epoch 48/50
8000/8000 - 0s - loss: 0.0028 - mean_squared_error: 0.0023 - mean_absolute_error: 0.0403 - val_loss: 0.0028 - val_mean_squared_error: 0.0023 - val_mean_absolute_error: 0.0393
Epoch 49/50
8000/8000 - 0s - loss: 0.0027 - mean_squared_error: 0.0021 - mean_absolute_error: 0.0385 - val_loss: 0.0026 - val_mean_squared_error: 0.0021 - val_mean_absolute_error: 0.0377
Epoch 50/50
8000/8000 - 1s - loss: 0.0025 - mean_squared_error: 0

Train on 8000 samples, validate on 2000 samples
Epoch 1/50
8000/8000 - 1s - loss: 0.2133 - mean_squared_error: 0.2132 - mean_absolute_error: 0.4337 - val_loss: 0.1977 - val_mean_squared_error: 0.1976 - val_mean_absolute_error: 0.4152
Epoch 2/50
8000/8000 - 1s - loss: 0.1870 - mean_squared_error: 0.1869 - mean_absolute_error: 0.4030 - val_loss: 0.1777 - val_mean_squared_error: 0.1776 - val_mean_absolute_error: 0.3907
Epoch 3/50
8000/8000 - 1s - loss: 0.1674 - mean_squared_error: 0.1673 - mean_absolute_error: 0.3804 - val_loss: 0.1582 - val_mean_squared_error: 0.1581 - val_mean_absolute_error: 0.3689
Epoch 4/50
8000/8000 - 1s - loss: 0.1481 - mean_squared_error: 0.1480 - mean_absolute_error: 0.3574 - val_loss: 0.1392 - val_mean_squared_error: 0.1391 - val_mean_absolute_error: 0.3452
Epoch 5/50
8000/8000 - 1s - loss: 0.1295 - mean_squared_error: 0.1293 - mean_absolute_error: 0.3334 - val_loss: 0.1213 - val_mean_squared_error: 0.1212 - val_mean_absolute_error: 0.3206
Epoch 6/50
8000/8000 -

Epoch 45/50
8000/8000 - 1s - loss: 0.0014 - mean_squared_error: 7.7089e-04 - mean_absolute_error: 0.0228 - val_loss: 0.0013 - val_mean_squared_error: 7.2994e-04 - val_mean_absolute_error: 0.0220
Epoch 46/50
8000/8000 - 1s - loss: 0.0013 - mean_squared_error: 7.0692e-04 - mean_absolute_error: 0.0217 - val_loss: 0.0013 - val_mean_squared_error: 6.7062e-04 - val_mean_absolute_error: 0.0209
Epoch 47/50
8000/8000 - 1s - loss: 0.0013 - mean_squared_error: 6.5083e-04 - mean_absolute_error: 0.0208 - val_loss: 0.0012 - val_mean_squared_error: 6.1218e-04 - val_mean_absolute_error: 0.0199
Epoch 48/50
8000/8000 - 1s - loss: 0.0012 - mean_squared_error: 5.9937e-04 - mean_absolute_error: 0.0198 - val_loss: 0.0012 - val_mean_squared_error: 5.6853e-04 - val_mean_absolute_error: 0.0192
Epoch 49/50
8000/8000 - 1s - loss: 0.0012 - mean_squared_error: 5.5210e-04 - mean_absolute_error: 0.0190 - val_loss: 0.0012 - val_mean_squared_error: 5.1909e-04 - val_mean_absolute_error: 0.0182
Epoch 50/50
8000/8000 - 1

INFO:tensorflow:Oracle triggered exit
539.5976529121399


In [8]:
# Build the model with the optimal hyperparameters and train it on the data for 25 epochs
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""
The hyperparameter search is complete:\n- The optimal learning rate for the optimizer is 
{best_hps.get('learning_rate')}\n- The optimal number of nodes in the hidden layer is {best_hps.get('hidden_units')}
""")
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_scale, Y_scale, epochs=100, shuffle=True, validation_split=0.2, verbose=2)

mean_squared_error_per_epoch = history.history['mean_squared_error']
best_epoch = mean_squared_error_per_epoch.index(min(mean_squared_error_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel_hist = hypermodel.fit(X_scale, Y_scale, epochs=best_epoch, shuffle=True, validation_split=0.2, verbose=2)


The hyperparameter search is complete:
- The optimal learning rate for the optimizer is 
0.001
- The optimal number of nodes in the hidden layer is 78

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 - 1s - loss: 0.1388 - mean_squared_error: 0.1387 - mean_absolute_error: 0.3395 - val_loss: 0.0721 - val_mean_squared_error: 0.0720 - val_mean_absolute_error: 0.2381
Epoch 2/100
8000/8000 - 1s - loss: 0.0421 - mean_squared_error: 0.0419 - mean_absolute_error: 0.1749 - val_loss: 0.0219 - val_mean_squared_error: 0.0216 - val_mean_absolute_error: 0.1264
Epoch 3/100
8000/8000 - 1s - loss: 0.0130 - mean_squared_error: 0.0126 - mean_absolute_error: 0.0967 - val_loss: 0.0078 - val_mean_squared_error: 0.0074 - val_mean_absolute_error: 0.0754
Epoch 4/100
8000/8000 - 1s - loss: 0.0056 - mean_squared_error: 0.0052 - mean_absolute_error: 0.0632 - val_loss: 0.0043 - val_mean_squared_error: 0.0039 - val_mean_absolute_error: 0.0537
Epoch 5/100
8000/8000 - 1s - loss: 0.0034 - mean_sq

Epoch 82/100
8000/8000 - 1s - loss: 9.3546e-04 - mean_squared_error: 1.9243e-04 - mean_absolute_error: 0.0104 - val_loss: 9.3423e-04 - val_mean_squared_error: 1.9191e-04 - val_mean_absolute_error: 0.0102
Epoch 83/100
8000/8000 - 1s - loss: 9.4627e-04 - mean_squared_error: 2.0431e-04 - mean_absolute_error: 0.0107 - val_loss: 9.3023e-04 - val_mean_squared_error: 1.8915e-04 - val_mean_absolute_error: 0.0101
Epoch 84/100
8000/8000 - 1s - loss: 9.3438e-04 - mean_squared_error: 1.9259e-04 - mean_absolute_error: 0.0104 - val_loss: 9.5644e-04 - val_mean_squared_error: 2.1437e-04 - val_mean_absolute_error: 0.0111
Epoch 85/100
8000/8000 - 1s - loss: 9.5433e-04 - mean_squared_error: 2.1386e-04 - mean_absolute_error: 0.0109 - val_loss: 9.2565e-04 - val_mean_squared_error: 1.8485e-04 - val_mean_absolute_error: 0.0100
Epoch 86/100
8000/8000 - 1s - loss: 9.3534e-04 - mean_squared_error: 1.9489e-04 - mean_absolute_error: 0.0105 - val_loss: 9.2794e-04 - val_mean_squared_error: 1.8545e-04 - val_mean_abs

Epoch 23/87
8000/8000 - 1s - loss: 9.5038e-04 - mean_squared_error: 2.0083e-04 - mean_absolute_error: 0.0107 - val_loss: 9.4048e-04 - val_mean_squared_error: 1.9060e-04 - val_mean_absolute_error: 0.0102
Epoch 24/87
8000/8000 - 1s - loss: 9.5390e-04 - mean_squared_error: 2.0372e-04 - mean_absolute_error: 0.0107 - val_loss: 9.6354e-04 - val_mean_squared_error: 2.1310e-04 - val_mean_absolute_error: 0.0108
Epoch 25/87
8000/8000 - 1s - loss: 9.5409e-04 - mean_squared_error: 2.0459e-04 - mean_absolute_error: 0.0108 - val_loss: 9.3734e-04 - val_mean_squared_error: 1.8757e-04 - val_mean_absolute_error: 0.0101
Epoch 26/87
8000/8000 - 1s - loss: 9.4336e-04 - mean_squared_error: 1.9324e-04 - mean_absolute_error: 0.0105 - val_loss: 9.3918e-04 - val_mean_squared_error: 1.8928e-04 - val_mean_absolute_error: 0.0103
Epoch 27/87
8000/8000 - 1s - loss: 9.5200e-04 - mean_squared_error: 2.0270e-04 - mean_absolute_error: 0.0107 - val_loss: 9.3340e-04 - val_mean_squared_error: 1.8466e-04 - val_mean_absolute

Epoch 64/87
8000/8000 - 1s - loss: 9.3632e-04 - mean_squared_error: 1.9519e-04 - mean_absolute_error: 0.0105 - val_loss: 9.2701e-04 - val_mean_squared_error: 1.8566e-04 - val_mean_absolute_error: 0.0102
Epoch 65/87
8000/8000 - 1s - loss: 9.4615e-04 - mean_squared_error: 2.0538e-04 - mean_absolute_error: 0.0108 - val_loss: 9.2796e-04 - val_mean_squared_error: 1.8771e-04 - val_mean_absolute_error: 0.0102
Epoch 66/87
8000/8000 - 1s - loss: 9.3700e-04 - mean_squared_error: 1.9631e-04 - mean_absolute_error: 0.0105 - val_loss: 9.2514e-04 - val_mean_squared_error: 1.8377e-04 - val_mean_absolute_error: 0.0101
Epoch 67/87
8000/8000 - 1s - loss: 9.3961e-04 - mean_squared_error: 1.9838e-04 - mean_absolute_error: 0.0106 - val_loss: 9.5405e-04 - val_mean_squared_error: 2.1309e-04 - val_mean_absolute_error: 0.0108
Epoch 68/87
8000/8000 - 1s - loss: 9.4686e-04 - mean_squared_error: 2.0620e-04 - mean_absolute_error: 0.0108 - val_loss: 9.3408e-04 - val_mean_squared_error: 1.9357e-04 - val_mean_absolute

In [9]:
# List the tuned hyperparameter values
# best_hps.__dict__.keys()
for key, value in best_hps.__dict__['values'].items():
    print(f'- {key}: {value}')

- hidden_units: 78
- learning_rate: 0.001
- tuner/epochs: 50
- tuner/initial_epoch: 0
- tuner/bracket: 0
- tuner/round: 0


In [10]:
file_path = '/Users/jackkelly/Desktop/Building_Model/Data_NEW/10000_initial_data_testing.csv'
X_test, Y_test = processing_data(N_channels, file_path)
eval_result = hypermodel.evaluate(X_test, Y_test, verbose=0)[1]
print(hypermodel.metrics_names)
print(f'MSE test: {eval_result}')

['loss', 'mean_squared_error', 'mean_absolute_error']
MSE test: 0.00018996083235833794


In [11]:
def visualising_data_MSE(hist, name):
#     MSE
    plt.title('Mean squared error vs. the number of epochs')
    plt.plot(hist.history['mean_squared_error'], label='Train')
    plt.plot(hist.history['val_mean_squared_error'], label='Validation')
    plt.ylabel('MSE')
    plt.xlabel('Epoch')
    plt.legend()
    plt.savefig(name, bbox_inches='tight', format='svg', dpi=1200)
    plt.show()

def visualising_data_MAE(hist, name):
    plt.title('Mean absolute error vs. the number of epochs')
    plt.plot(hist.history['mean_absolute_error'], label='Train')
    plt.plot(hist.history['val_mean_absolute_error'], label='Validation')
    plt.ylabel('MAE')
    plt.xlabel('Epoch')
    plt.legend()
    plt.savefig(name, bbox_inches='tight', format='svg', dpi=1200)
    plt.show()
    
# data up to the best epoch
# visualising_data_MSE(hypermodel_hist, '/Users/jackkelly/Desktop/Building_Model/Figures/hypermodel_results_MSE.svg')
# data for the full 100 epochs
visualising_data_MSE(history, '/Users/jackkelly/Desktop/Building_Model/Figures/full_epochs_results_MSE_FPGA.svg')
visualising_data_MAE(history, '/Users/jackkelly/Desktop/Building_Model/Figures/full_epochs_results_MAE_FPGA.svg')

/opt/anaconda3/envs/project/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  # Remove the CWD from sys.path while we load stuff.
/opt/anaconda3/envs/project/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


In [12]:
# shows weights of the ANN
# print(hypermodel.weights)
hypermodel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 78)                3276      
_________________________________________________________________
dense_1 (Dense)              (None, 40)                3160      
Total params: 6,436
Trainable params: 6,436
Non-trainable params: 0
_________________________________________________________________


In [13]:
# save the tuned model
hypermodel.save('/Users/jackkelly/Desktop/Building_Model/Tuned_40_channels_ANN_model_FPGA.h5')

In [14]:
# necessary to save this temporary file as don't to include the optimizer as that wouldn't actually be part of the ANN
_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(hypermodel, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

Saved baseline model to: /var/folders/yp/nbwcv8453t3f3sx3xxwwycqm0000gn/T/tmpvrp53eyk.h5


# Low-magnitude Pruning with Quantization

In [24]:
# CSV file manipulation

def instantiateWriteFile(name):
    with open(name, 'w', newline='') as file:
        writer = csv.writer(file)
        # 51 input variables in total
        labels = [0.2, 0.4, 0.6, 0.8]
        writer.writerow(labels)
        
def add_row_of_data(file_path, list_of_elem):
    with open(file_path, 'a+', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(list_of_elem)

In [25]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
    import zipfile

    _, zipped_file = tempfile.mkstemp('.zip')
    with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
        f.write(file)

    return os.path.getsize(zipped_file)

In [76]:
instantiateWriteFile('/Users/jackkelly/Desktop/Building_Model/Data_NEW/sparsity_MSE_data_v2.csv')
instantiateWriteFile('/Users/jackkelly/Desktop/Building_Model/Data_NEW/sparsity_size_data_v2.csv')

file_path = '/Users/jackkelly/Desktop/Building_Model/Data_NEW/10000_initial_data_testing.csv'
X_test, Y_test = processing_data(N_channels, file_path)
model = keras.models.load_model('/Users/jackkelly/Desktop/Building_Model/Tuned_40_channels_ANN_model_FPGA.h5')
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
batch_size = 32
epochs = 100
validation_split = 0.2

In [80]:
def prune_and_quantize(desired_sparsity, model):
    # quantized_bits(no., integer bits, symmetric)
    config = {
      "QDense": {
          "kernel_quantizer": "quantized_bits(12,0,1)",
          "bias_quantizer": "quantized_bits(12)"
      },
      "QActivation": { "relu": "quantized_bits(12)" },
    }
    qmodel = model_quantize(model, config, 16, transfer_weights=True)
    pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(target_sparsity=desired_sparsity, begin_step=0, end_step=-1, frequency=100)
    }

    # take the 1 layer hyermodel and prune it
    pruned_q_model = prune_low_magnitude(qmodel, **pruning_params)
    # 'prune_low_magnitude' requires a recompile.
    pruned_q_model.compile(optimizer=keras.optimizers.Adam(),
                        loss=keras.losses.MeanSquaredError(), metrics=['mean_squared_error'])
    callbacks = [
      tfmot.sparsity.keras.UpdatePruningStep(),
    ]
    pruned_q_model.fit(X_scale, Y_scale,
                      batch_size=batch_size, epochs=epochs, validation_split=validation_split, callbacks=callbacks, verbose=2)
    # evaluate performance
    eval_pruned_model = pruned_q_model.evaluate(X_test, Y_test, verbose=0)[1]
    print(f'Pruned model MSE: {eval_pruned_model}')
    qmodel_stripped = tfmot.sparsity.keras.strip_pruning(pruned_q_model)
    _, pruned_keras_file = tempfile.mkstemp('.h5')
    tf.keras.models.save_model(qmodel_stripped, pruned_keras_file, include_optimizer=False)
    print('Saved pruned Keras model to:', pruned_keras_file)
    # Size of model after pruning
    size = get_gzipped_model_size(pruned_keras_file)
    print("Size of gzipped baseline Keras model: %.2f bytes" % (size))
    return eval_pruned_model, size

eval_pruned_results = []
size_results = []
pruning_list = [0.2, 0.4, 0.6, 0.8]
for i in pruning_list:
    eval_pruned_model, size = prune_and_quantize(i, model)
    eval_pruned_results.append(eval_pruned_model)
    size_results.append(size)

file_path = '/Users/jackkelly/Desktop/Building_Model/Data_NEW/sparsity_MSE_data_v2.csv'
add_row_of_data(file_path, eval_pruned_results)
file_path = '/Users/jackkelly/Desktop/Building_Model/Data_NEW/sparsity_size_data_v2.csv'
add_row_of_data(file_path, size_results)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 - 6s - loss: 0.0013 - mean_squared_error: 6.0651e-04 - val_loss: 0.0011 - val_mean_squared_error: 3.6104e-04
Epoch 2/100
8000/8000 - 2s - loss: 0.0013 - mean_squared_error: 5.2982e-04 - val_loss: 0.0013 - val_mean_squared_error: 5.5668e-04
Epoch 3/100
8000/8000 - 2s - loss: 0.0012 - mean_squared_error: 4.7472e-04 - val_loss: 0.0011 - val_mean_squared_error: 3.0377e-04
Epoch 4/100
8000/8000 - 2s - loss: 0.0011 - mean_squared_error: 3.6908e-04 - val_loss: 0.0011 - val_mean_squared_error: 3.1928e-04
Epoch 5/100
8000/8000 - 2s - loss: 0.0011 - mean_squared_error: 3.1907e-04 - val_loss: 0.0011 - val_mean_squared_error: 3.0291e-04
Epoch 6/100
8000/8000 - 2s - loss: 0.0010 - mean_squared_error: 2.8699e-04 - val_loss: 0.0011 - val_mean_squared_error: 3.3083e-04
Epoch 7/100
8000/8000 - 2s - loss: 0.0011 - mean_squared_error: 3.2052e-04 - val_loss: 0.0011 - val_mean_squared_error: 3.0422e-04
Epoch 8/100
8000/8000 - 2s - loss: 

Epoch 63/100
8000/8000 - 2s - loss: 0.0010 - mean_squared_error: 2.7322e-04 - val_loss: 0.0014 - val_mean_squared_error: 6.1103e-04
Epoch 64/100
8000/8000 - 2s - loss: 0.0010 - mean_squared_error: 2.7892e-04 - val_loss: 9.9664e-04 - val_mean_squared_error: 2.3449e-04
Epoch 65/100
8000/8000 - 2s - loss: 0.0010 - mean_squared_error: 2.6011e-04 - val_loss: 0.0010 - val_mean_squared_error: 2.6203e-04
Epoch 66/100
8000/8000 - 2s - loss: 0.0011 - mean_squared_error: 2.9193e-04 - val_loss: 0.0010 - val_mean_squared_error: 2.5254e-04
Epoch 67/100
8000/8000 - 2s - loss: 0.0010 - mean_squared_error: 2.4471e-04 - val_loss: 9.8606e-04 - val_mean_squared_error: 2.2307e-04
Epoch 68/100
8000/8000 - 2s - loss: 0.0010 - mean_squared_error: 2.3751e-04 - val_loss: 0.0010 - val_mean_squared_error: 2.6234e-04
Epoch 69/100
8000/8000 - 2s - loss: 0.0010 - mean_squared_error: 2.5534e-04 - val_loss: 0.0010 - val_mean_squared_error: 2.8003e-04
Epoch 70/100
8000/8000 - 2s - loss: 0.0010 - mean_squared_error: 2.5

Epoch 23/100
8000/8000 - 2s - loss: 0.0010 - mean_squared_error: 2.3495e-04 - val_loss: 9.6977e-04 - val_mean_squared_error: 1.9865e-04
Epoch 24/100
8000/8000 - 2s - loss: 9.9888e-04 - mean_squared_error: 2.2774e-04 - val_loss: 0.0010 - val_mean_squared_error: 2.4242e-04
Epoch 25/100
8000/8000 - 2s - loss: 0.0010 - mean_squared_error: 2.4246e-04 - val_loss: 9.5998e-04 - val_mean_squared_error: 1.8897e-04
Epoch 26/100
8000/8000 - 2s - loss: 0.0010 - mean_squared_error: 2.3402e-04 - val_loss: 0.0011 - val_mean_squared_error: 3.0146e-04
Epoch 27/100
8000/8000 - 2s - loss: 0.0010 - mean_squared_error: 2.3953e-04 - val_loss: 0.0010 - val_mean_squared_error: 2.5017e-04
Epoch 28/100
8000/8000 - 2s - loss: 0.0010 - mean_squared_error: 2.5664e-04 - val_loss: 0.0010 - val_mean_squared_error: 2.5133e-04
Epoch 29/100
8000/8000 - 2s - loss: 0.0011 - mean_squared_error: 2.9570e-04 - val_loss: 0.0011 - val_mean_squared_error: 2.9334e-04
Epoch 30/100
8000/8000 - 2s - loss: 0.0011 - mean_squared_error:

Epoch 44/100
8000/8000 - 2s - loss: 9.7442e-04 - mean_squared_error: 1.9072e-04 - val_loss: 9.6786e-04 - val_mean_squared_error: 1.8508e-04
Epoch 45/100
8000/8000 - 2s - loss: 9.7358e-04 - mean_squared_error: 1.9123e-04 - val_loss: 9.6591e-04 - val_mean_squared_error: 1.8341e-04
Epoch 46/100
8000/8000 - 2s - loss: 9.7258e-04 - mean_squared_error: 1.9041e-04 - val_loss: 9.6551e-04 - val_mean_squared_error: 1.8354e-04
Epoch 47/100
8000/8000 - 2s - loss: 9.7073e-04 - mean_squared_error: 1.8948e-04 - val_loss: 9.6597e-04 - val_mean_squared_error: 1.8486e-04
Epoch 48/100
8000/8000 - 2s - loss: 9.6993e-04 - mean_squared_error: 1.8979e-04 - val_loss: 9.6712e-04 - val_mean_squared_error: 1.8694e-04
Epoch 49/100
8000/8000 - 2s - loss: 9.6854e-04 - mean_squared_error: 1.8856e-04 - val_loss: 9.7068e-04 - val_mean_squared_error: 1.9099e-04
Epoch 50/100
8000/8000 - 2s - loss: 9.6862e-04 - mean_squared_error: 1.8939e-04 - val_loss: 9.6395e-04 - val_mean_squared_error: 1.8567e-04
Epoch 51/100
8000/80

Epoch 67/100
8000/8000 - 2s - loss: 0.0034 - mean_squared_error: 0.0028 - val_loss: 0.0035 - val_mean_squared_error: 0.0028
Epoch 68/100
8000/8000 - 2s - loss: 0.0034 - mean_squared_error: 0.0027 - val_loss: 0.0034 - val_mean_squared_error: 0.0028
Epoch 69/100
8000/8000 - 2s - loss: 0.0033 - mean_squared_error: 0.0027 - val_loss: 0.0034 - val_mean_squared_error: 0.0028
Epoch 70/100
8000/8000 - 2s - loss: 0.0033 - mean_squared_error: 0.0027 - val_loss: 0.0034 - val_mean_squared_error: 0.0028
Epoch 71/100
8000/8000 - 2s - loss: 0.0033 - mean_squared_error: 0.0027 - val_loss: 0.0034 - val_mean_squared_error: 0.0028
Epoch 72/100
8000/8000 - 2s - loss: 0.0033 - mean_squared_error: 0.0026 - val_loss: 0.0033 - val_mean_squared_error: 0.0027
Epoch 73/100
8000/8000 - 2s - loss: 0.0032 - mean_squared_error: 0.0026 - val_loss: 0.0033 - val_mean_squared_error: 0.0027
Epoch 74/100
8000/8000 - 2s - loss: 0.0032 - mean_squared_error: 0.0026 - val_loss: 0.0033 - val_mean_squared_error: 0.0027
Epoch 75

# Analysing Data

In [81]:
df = pd.read_csv('/Users/jackkelly/Desktop/Building_Model/Data_NEW/sparsity_MSE_data_v2.csv')
MSE_values = df.values
MSE_values

array([[0.00022777, 0.00020082, 0.00018896, 0.00228342],
       [0.00023086, 0.00025096, 0.00018886, 0.0023179 ]])

In [82]:
df = pd.read_csv('/Users/jackkelly/Desktop/Building_Model/Data_NEW/sparsity_size_data_v2.csv')
size_values = df.values
size_values

array([[22635, 18901, 14640,  9134],
       [22638, 18935, 14632,  9066]])

In [64]:
plt.figure(0)
plt.title('Mean squared error vs. the number of epochs')
plt.plot(pruning_list, MSE_values[0,:])
# plt.plot(hist.history['val_mean_squared_error'], label='Validation')
plt.ylabel('MSE')
plt.xlabel('Sparsity')
# plt.legend()
name = '/Users/jackkelly/Desktop/Building_Model/Figures/sparsity_plot.svg'
plt.savefig(name, bbox_inches='tight', format='svg', dpi=1200)
plt.show()

/opt/anaconda3/envs/project/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  # Remove the CWD from sys.path while we load stuff.


In [65]:
plt.figure(1)
plt.title('Mean squared error vs. the number of epochs')
plt.plot(pruning_list, size_values[0,:])
# plt.plot(hist.history['val_mean_squared_error'], label='Validation')
plt.ylabel('MSE')
plt.xlabel('Sparsity')
# plt.legend()
name = '/Users/jackkelly/Desktop/Building_Model/Figures/size_plot.svg'
plt.savefig(name, bbox_inches='tight', format='svg', dpi=1200)
plt.show()

/opt/anaconda3/envs/project/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  # Remove the CWD from sys.path while we load stuff.
